In [1]:
# import parameters 
from parameters import *

## Read the polygons file 

In [2]:
import os 

polygon_file = os.path.expanduser(polygon_file)

# verify that the file existe 
print(os.path.isfile(polygon_file))

True


## Read the polygons file 

In [6]:
# insert the polygons in a geopandas dataframe
import geopandas as gdp 

polygons = gdp.read_file(polygon_file)
polygons.head()

,YEAR_,RESERVE,Ownership,Shape_Leng,Shape_Area,SP_Name,Sp_class,water_mgtZ,Name,layer,id,geometry
0,2006,Bujawe,NFA,2742.477747,271665.183923,Eucalyptus grandis,Broadleaved,Albert Water Management Zone,BujaweNFA,NFA_Plantations_Jan2021,1,"POLYGON ((31.19657 1.53990, 31.19414 1.54045, ..."
1,2017,Kaweri,NFA,4297.979169,625614.849598,Eucalyptus grandis,Broadleaved,Victoria Water Management Zo,Kaweri NFA,NFA_Plantations_Jan2021,2,"POLYGON ((31.43501 0.60211, 31.43504 0.60209, ..."
2,2017,Mafuga,Private,3938.167795,808056.323881,Eucalyptus grandis,Broadleaved,Albert Water Management Zone,Mafuga Private,Private_Plantations_Jan2021,3,"POLYGON ((29.85428 -1.01405, 29.85429 -1.01405..."
3,2005,Rwoho,NFA,6001.759500,878570.695436,Pinus caribaea,Coniferous,Victoria Water Management Zo,Rwoho NFA,NFA_Plantations_Jan2021,4,"POLYGON ((30.54939 -0.83256, 30.54947 -0.83323..."
4,1986,Nandagi,NFA,3418.832619,489515.000001,Terminalia sp.,Broadleaved,Kyoga Water Management Zone,Nandagi NFA,NFA_Plantations_Jan2021,5,"POLYGON ((32.82506 0.38618, 32.82490 0.38627, ..."


In [7]:
# only select the specific one (if test mode)
if test_mode:
    polygons = polygons[polygons['id'] < 3]
polygons

,YEAR_,RESERVE,Ownership,Shape_Leng,Shape_Area,SP_Name,Sp_class,water_mgtZ,Name,layer,id,geometry
0,2006,Bujawe,NFA,2742.477747,271665.183923,Eucalyptus grandis,Broadleaved,Albert Water Management Zone,BujaweNFA,NFA_Plantations_Jan2021,1,"POLYGON ((31.19657 1.53990, 31.19414 1.54045, ..."
1,2017,Kaweri,NFA,4297.979169,625614.849598,Eucalyptus grandis,Broadleaved,Victoria Water Management Zo,Kaweri NFA,NFA_Plantations_Jan2021,2,"POLYGON ((31.43501 0.60211, 31.43504 0.60209, ..."


In [8]:
# transform the polygon into a square
from utils import *

squares = polygons.copy()
squares['geometry'] = squares['geometry'].map(to_square)

squares.head()

,YEAR_,RESERVE,Ownership,Shape_Leng,Shape_Area,SP_Name,Sp_class,water_mgtZ,Name,layer,id,geometry
0,2006,Bujawe,NFA,2742.477747,271665.183923,Eucalyptus grandis,Broadleaved,Albert Water Management Zone,BujaweNFA,NFA_Plantations_Jan2021,1,"POLYGON ((31.20757 1.54930, 31.20757 1.53128, ..."
1,2017,Kaweri,NFA,4297.979169,625614.849598,Eucalyptus grandis,Broadleaved,Victoria Water Management Zo,Kaweri NFA,NFA_Plantations_Jan2021,2,"POLYGON ((31.44150 0.60583, 31.44150 0.58782, ..."


In [9]:
# create a list of ee object based on the shapely figures in geopandas
import ee

ee.Initialize() 

# decide not to use a lis comprehension for readability 
ee_squares = []
for index, row in squares.iterrows():
    ee_square = ee.Geometry.Polygon(list(row['geometry'].exterior.coords))
    ee_squares.append(ee_square)
    
# add these ee_polygons to he dataset 
squares['ee_geometry'] = ee_squares
squares.head()

,YEAR_,RESERVE,Ownership,Shape_Leng,Shape_Area,SP_Name,Sp_class,water_mgtZ,Name,layer,id,geometry,ee_geometry
0,2006,Bujawe,NFA,2742.477747,271665.183923,Eucalyptus grandis,Broadleaved,Albert Water Management Zone,BujaweNFA,NFA_Plantations_Jan2021,1,"POLYGON ((31.20757 1.54930, 31.20757 1.53128, ...","ee.Geometry({\n ""functionInvocationValue"": {\..."
1,2017,Kaweri,NFA,4297.979169,625614.849598,Eucalyptus grandis,Broadleaved,Victoria Water Management Zo,Kaweri NFA,NFA_Plantations_Jan2021,2,"POLYGON ((31.44150 0.60583, 31.44150 0.58782, ...","ee.Geometry({\n ""functionInvocationValue"": {\..."


In [10]:
# check if the result is effectively a square
from sepal_ui import mapping as sm 
import geemap

m = sm.SepalMap(['CartoDB.Positron'])

# displays the first nb_squares squares
for index, row in squares.iterrows():
    m.addLayer(row['ee_geometry'], {'color':'red'}, row['Name'])
    if index == 0:
        m.centerObject(row['ee_geometry'], zoom=14)
    if index > min(len(squares), nb_squares):
        break
m

Styles()

SepalMap(center=[1.5402918372065977, 31.198558429906072], controls=(ZoomControl(options=['position', 'zoom_in_…

## Create the tif file in earthengine 

In [11]:
from sepal_ui import sepalwidgets as sw

output = sw.Alert()
output.show()

Alert(class_='mt-5', text=True, type='info')

In [12]:
from scripts import *

path = createPDF(polygon_file, squares, polygons, bands_combo, sources, output)
sw.DownloadBtn('clickable link', path)

DownloadBtn(children=[Icon(children=['mdi-download'], left=True), 'clickable link'], class_='ma-2', color='suc…